# 1. Preparations

### 1-1. Import Libraries
- 데이터셋 다운로드와 전처리를 쉽게 하는 torchtext 라이브러리를 import 합니다.


In [1]:
import os
import random
import time
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import data, datasets
import random
import time
import spacy
import numpy as np
from torch import Tensor

### 1-2. Load data
- Field 를 정의합니다.
- IMDB 데이터를 다운받습니다.
- Train,valid,test 데이터셋으로 split 합니다.

In [2]:
TEXT = data.Field(tokenize = 'spacy', include_lengths=True)
LABEL = data.LabelField(dtype = torch.float) 

In [3]:
# Download IMDB data (about 3mins)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 10.8MB/s]


In [4]:
# Set the random seed
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
# Split train and valid data
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print('Number of training examples: {}'.format(len(train_data)))
print('Number of validation examples: {}'.format(len(valid_data)))
print('Number of testing examples: {}'.format(len(test_data)))

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [7]:
# print example
print(vars(train_data.examples[0]))
print(' '.join(vars(train_data.examples[0])['text']))

{'text': ['You', 'know', 'what', 'you', 'are', 'getting', 'when', 'you', 'purchase', 'a', 'Hallmark', 'card', '.', 'A', 'sappy', ',', 'trite', 'verse', 'and', 'that', 'will', 'be', '$', '3.99', ',', 'thank', 'you', 'very', 'much', '.', 'You', 'get', 'the', 'same', 'with', 'a', 'Hallmark', 'movie', '.', 'Here', 'we', 'get', 'a', 'ninety', 'year', 'old', 'Ernie', 'Borgnine', 'coming', 'out', 'of', 'retirement', 'to', 'let', 'us', 'know', 'that', 'as', 'a', 'matter', 'of', 'fact', ',', 'he', 'is', 'not', 'dead', 'like', 'we', 'thought', '.', 'Poor', 'Ernie', ',', 'he', 'is', 'the', 'poor', 'soul', 'that', 'married', 'Ethel', 'Merman', 'several', 'years', 'ago', 'and', 'the', 'marriage', 'lasted', 'a', 'few', 'weeks', '.', 'In', 'this', 'flick', ',', 'Ernie', 'jumps', 'in', 'feet', 'first', 'and', 'portrays', 'the', 'Grandpa', 'that', 'bonds', 'with', 'his', 'long', 'lost', 'grandkid', '.', 'We', 'have', 'seen', 'it', 'before', '.', 'You', 'might', 'enjoy', 'this', 'movie', 'but', 'please'

### 1-3. Cuda Setup
- GPU 사용을 위한 Cuda 설정
- Colab 페이지 상단 메뉴>수정>노트설정에서 GPU 사용 설정이 선행되어야 합니다.


In [8]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda:0" if USE_CUDA else "cpu")
print(device)

cuda:0


In [9]:
!nvidia-smi

Tue Nov 10 11:56:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##2. Preprocess data
- Vocab (단어장) 을 만듭니다.
- Iterator 를 만듭니다. (Iterator 를 통해 batch training 을 위한 batching 과 padding, 그리고 데이터 내 단어들의 인덱스 변환이 이루어집니다.)  

In [10]:
# Load pre-trained word vectors (about 7mins)
TEXT.build_vocab(train_data, vectors = "glove.6B.100d")

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 399714/400000 [00:23<00:00, 17295.48it/s]

In [11]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")

Unique tokens in TEXT vocabulary: 101446


In [12]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_                 
                 )
LABEL.build_vocab(train_data)

In [13]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [14]:
TEXT.vocab.itos[:10]  #itos – A list of token strings indexed by their numerical identifiers.

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']

In [15]:
word_dict = TEXT.vocab.stoi # stoi – A collections.defaultdict instance mapping token strings to numerical identifiers.
print(len(word_dict))
print(word_dict['<unk>'], word_dict['<pad>'])
print(list(word_dict)[-1], word_dict[list(word_dict)[-1]])

25002
0 1
buffaloes 25001


In [16]:
print(TEXT.vocab.vectors.shape)
print(TEXT.vocab.vectors)

torch.Size([25002, 100])
tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3556, -0.2554, -0.1192,  ..., -0.1305,  0.1196, -0.4377],
        [-0.5618,  0.5658, -0.2214,  ..., -0.2283,  0.4419,  0.5798],
        [ 0.3757,  0.3498,  0.7955,  ...,  0.7336,  0.2335, -0.1470]])


In [17]:
# Batching - construct iterator
BATCH_SIZE = 32   

train_iterator = data.Iterator(
    train_data, 
    batch_size = BATCH_SIZE,
    device = device)

# shape: BATCH_SIZE x maximum length of sentence 

for batch in train_iterator:
    break
print(batch.text)
print(len(batch.text[0]))
print(batch.label)

(tensor([[   66,  2644,  2739,  ...,   421,    66,    66],
        [  671,    39, 14358,  ...,   752,     9,   293],
        [   97,   976,    10,  ...,   960,  1053,    10],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:0'), tensor([187, 171, 173, 239, 253, 157, 161, 367, 130, 128, 239, 301, 287, 617,
        570, 347, 261, 384, 147, 181, 164, 159, 188, 230,  64, 176, 260, 592,
        342, 187, 233, 332], device='cuda:0'))
617
tensor([0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
       device='cuda:0')


In [18]:
# BucketIterator

train_iterator = data.BucketIterator(
    train_data, 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

for batch in train_iterator:
    break
print(batch.text)
print(len(batch.text[0]))
print(batch.label)

(tensor([[   25,    66,    66,  ...,    27,    66,    11],
        [   22,    24,    24,  ...,     0,   426,   158],
        [   19,     9,     9,  ...,    18,  1789,   124],
        ...,
        [  807, 23936,  3698,  ...,     0,    67,     4],
        [  711,   117,    24,  ...,     1,     1,     1],
        [    4,     4,     4,  ...,     1,     1,     1]], device='cuda:0'), tensor([133, 133, 133, 133, 133, 133, 133, 133, 133, 133, 133, 133, 133, 133,
        132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 132, 131, 131, 131,
        131, 131, 131, 131], device='cuda:0'))
133
tensor([0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
        0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1.],
       device='cuda:0')


In [19]:
# Batching - construct iterator

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_sizes = (BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),
    sort_within_batch = True,
    device = device)

##3. Build Model
- Embedding layer, RNN layer, Dropout layer, Fully-connected layer 로 이루어진 모델을 만듭니다.
- 미리 학습된 워드 임베딩을 임베딩 레이어에 올립니다.

In [214]:
class Model(nn.Module):  # Custom model 정의 
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()

        # Define parameters
        self.hidden_him = hidden_dim
        self.n_layers = n_layers

        # Define Layers
        # Embedding layer
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)

        # RNN layer
        # self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, bidirectional=bidirectional, dropout=dropout)
        # self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional=bidirectional, dropout=dropout)
        self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, bidirectional=bidirectional, dropout=dropout)

        # Fully connected layer
        #self.fc = nn.Linear(hidden_dim, output_dim)
        self.fc = nn.Linear(hidden_dim * 2, output_dim) # hidden_dim *2 인 이유? bidirectional이기 때문

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        
    def forward(self, text):

        # text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        # embedded = [sent len, batch size, emb dim]
        # embedded = [batch size, sent len, emb dim] if batch_first = True
        
        # output, hidden = self.rnn(embedded)
        # output, (hidden, cell) = self.lstm(embedded)
        output, hidden = self.gru(embedded)


        # hidden = [num_layers x num_directions, batch_size, hidden_size]
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        return self.fc(hidden.squeeze(0))




---

## (추가설명)
### Bidirectional RNN 의 "concatenation" 에 대하여
* `batch_size = 3, hidden_dim = 10, n_layers = 1, bidirectional = True` 일때 
* RNN 모델은 forward layer 와 backward layer 총 2개 레이어를 가지게 됩니다.
* 편의상 forward layer 의 hidden state 의 모든 unit 이 0이 되고,
backward layer 의 경우 모두 1이 된다고 가정하겠습니다.


In [215]:
# 한개의 input 이 들어왔을때, 마지막 타임 스텝에서의 forward/backward hidden state 는 각각 다음과 같은 형태가 될 것입니다.
h_forward = torch.zeros(1,10) 
h_backward = torch.ones(1,10)
print(h_forward)
print(h_backward)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


 


*   Torch.nn 제공 RNN 모듈은 2개의 아웃풋 중 하나로 hidden state 을 출력하며,
> `output, hidden = self.rnn(embedded)`
*   `hidden`은 모델에 들어있는 **모든 레이어**의 last hidden state 을 출력합니다.
*   따라서 `hidden` 의 형태는 `[num_layers x num_directions, batch_size, hidden_size]`가 됩니다.

* 모델에서 총 n개의 layer 를 사용할 경우, 순서대로 _1번째 forward, 1번째 backward, 2번째 forward, 2번째 backward, ..., n번째 forward, n번째 backward_ 가 표시됩니다.
* 이 예제에서는 `n_layers = 1`이므로 `hidden` 의 shape 은 `[2,3,10]` 이 됩니다.

In [216]:
# hidden 은 아래와 같이 생기게 됩니다.
hidden = torch.cat([h_forward.unsqueeze(0).repeat([1,3,1]),h_backward.unsqueeze(0).repeat([1,3,1])])

print(hidden)
print(hidden.shape)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]])
torch.Size([2, 3, 10])



*   우리는 forward 와 backward layer 각각에서 나온 last hidden state를 나란히 합치고자 합니다.
*  `hidden[-2,:,:]` 는 forward layer 의 last hidden state 을 나타내고
*  `hidden[-1,:,:]` 는 backward layer 의 last hidden state 을 나타냅니다.
* 이 두개를 hidden_size 를 나타내는 dimension=1 의 방향으로 concatenate 합니다.




In [217]:
# 최종적으로 사용하는 Bidirectional RNN 모델의 아웃풋은 다음과 같은 형태를 가집니다.
h_concat = torch.cat([hidden[-2,:,:],hidden[-1,:,:]],dim=1)
print(h_concat)
print(h_concat.shape)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.]])
torch.Size([3, 20])


## (추가설명 종료)
---


In [218]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
# HIDDEN_DIM = 128
OUTPUT_DIM = 1
N_LAYERS = 2
# N_LAYERS = 1
# N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = Model(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)    # Make model instance


In [219]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)  # Count number of elements of all parameters

print('The model has {:,} trainable parameters'.format(count_parameters(model)))

The model has 4,233,321 trainable parameters


In [220]:
# load pretrained embeddings
pretrained_embeddings = TEXT.vocab.vectors
print(type(pretrained_embeddings))
model.embedding.weight.data.copy_(pretrained_embeddings);

<class 'torch.Tensor'>


## 4. Train model

In [221]:
optimizer = optim.Adam(model.parameters())   # Gradient Descent 실행

In [222]:
criterion = nn.BCEWithLogitsLoss()  # 손실함수 정의 

In [223]:
model = model.to(device)  #모델을 GPU 로 이동
criterion = criterion.to(device)

In [224]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [225]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()  # Gradient 0으로 초기화
                
        predictions = model(batch.text[0]).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()    # backward pass (gradient 계산)
        
        optimizer.step()   # parameter update
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [226]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            predictions = model(batch.text[0]).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


### *Do Training!*

In [227]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'rnn-model.pt')
    
    print('Epoch: {:02}'.format(epoch+1))
    print('\tTrain Loss: {:.3f} | Train Acc: {:.2f}%'.format(train_loss, train_acc*100))
    print('\t Val. Loss: {:.3f} |  Val. Acc: {:.2f}%'.format(valid_loss, valid_acc*100))

Epoch: 01
	Train Loss: 0.529 | Train Acc: 70.77%
	 Val. Loss: 0.399 |  Val. Acc: 83.57%
Epoch: 02
	Train Loss: 0.244 | Train Acc: 90.44%
	 Val. Loss: 0.335 |  Val. Acc: 85.94%
Epoch: 03
	Train Loss: 0.144 | Train Acc: 94.93%
	 Val. Loss: 0.308 |  Val. Acc: 88.69%
Epoch: 04
	Train Loss: 0.077 | Train Acc: 97.48%
	 Val. Loss: 0.331 |  Val. Acc: 89.11%
Epoch: 05
	Train Loss: 0.034 | Train Acc: 98.96%
	 Val. Loss: 0.435 |  Val. Acc: 88.46%


In [228]:
model.load_state_dict(torch.load('rnn-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print('Test Loss: {:.3f} | Test Acc: {:.2f}%'.format(test_loss, test_acc*100))

Test Loss: 0.312 | Test Acc: 88.59%


## 5. Test model
우리가 직접 예문을 작성해서 트레인된 모델에서 예문을 어떻게 평가하는지 확인합니다.



In [229]:
# 토크나이저로 spacy 를 사용합니다.
nlp = spacy.load('en')

# 사용자가 입력한 sentence 를 훈련된 모델에 넣었을때의 결과값을 확인합니다.
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  # Tokenization
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]   # 위에서 만든 vocab 에 부여된 index 로 indexing
    tensor = torch.LongTensor(indexed).to(device)   # indexing 된 sequence 를 torch tensor 형태로 만들어줌.
    tensor = tensor.unsqueeze(1)   # 입력 텐서에 batch 차원을 만들어줌.
    prediction = torch.sigmoid(model(tensor))  # 모델에 입력한 후 확률값 도출을 위한 sigmoid 적용 
    return prediction.item() # prediction 값 출력

In [230]:
predict_sentiment(model, "This film is terrible") #아주 낮은 값의 확률이 도출되는 것을 확인할 수 있습니다.(부정)

0.0011325414525344968

In [231]:
predict_sentiment(model, "This film is great") #아주 높은 값의 확률이 도출되는 것을 확인할 수 있습니다. (긍정)

0.9992702603340149